# **Diplomado IA: Aplicaciones de Inteligencia Artificial I - Parte 2**. <br> Práctico 4: Generación de resúmenes
---
---

**Profesores:**
- Vladimir Araujo
- Felipe del Río

**Ayudante:**
- Nissim Ergas
---
---

# **Instrucciones Generales**

El siguiente práctico se debe realizar de forma **individual**. El formato de entregar es el **archivo .ipynb con todas las celdas ejecutadas**. Todas las preguntas deben ser respondida en celdas de texto. No se aceptará el _output_ de una celda de código como respuesta.

**Nombre:** FRANCISCO MENA

**Fecha de entrega: Viernes 8 de enero.**

El siguiente práctico cuanta con 2 secciones donde cada una contendrá 1 o más actividades a realizar. Algunas actividades correspondrán a escribir código y otras a responder preguntas. 

Se recomienda **fuertemente** revisar las secciones donde se entrega código porque algunas actividades de código pueden reutilizar el mismo código pero con cambios en algunas líneas.



# **Agenda**

>[Diplomado IA: Aplicaciones de Inteligencia Artificial I - Parte 2.  Práctico 4: Generación de resúmenes](#scrollTo=tHopPtVaNF1K)

>[Instrucciones Generales](#scrollTo=uIdAKAdELPSl)

>[Agenda](#scrollTo=kEloa5uXLIPK)

>[Parte II: Generación de Resumenes Abstractivos ft. OpenNMT](#scrollTo=I3GBi78rnDo_)

>>[Preámbulo](#scrollTo=jFqmDr1T6mVq)

>>[Creación del vocabulario](#scrollTo=WHBf45mu25ya)

>>[Entrenamiento](#scrollTo=TJGWJOFz3Pnh)

>>[Creación de Resumenes](#scrollTo=SCJz6aHv3GY8)

>>>[Preámbulo](#scrollTo=SCJz6aHv3GY8)

>>>[Modelo Base](#scrollTo=zYaRvQ-lzTN6)

>>>>[Configuración de la ejecución](#scrollTo=zYaRvQ-lzTN6)

>>>>[Evaluación cualitativa](#scrollTo=E8MyjKOkU0oR)

>>>>[Evaluación cuantitativa](#scrollTo=D3tKPzA9Jbm7)

>>>[Modelo Pointer](#scrollTo=-O1CWc8rLt1n)

>>>>[Evaluación cualitativa](#scrollTo=BBhZbHUQWkGA)

>>>>[Evaluación cuantitativa](#scrollTo=f32nJ1iQWkGN)

>>>>[Actividad 2](#scrollTo=qMaayH1yiMup)

>>>[Actividad 3: Modelo Coverage](#scrollTo=O2kVqHVELyGR)

>>>>[Evaluación cualitativa](#scrollTo=UAMSGfksWueT)

>>>>[Evaluación cuantitativa](#scrollTo=E2QFB_-5Wuee)

>>>>[Respuesta Actividad 3](#scrollTo=EKL2gbIyVHRG)



# Parte II: Generación de Resumenes Abstractivos ft. OpenNMT

En esta segunda parte del laboratorio, utilizaremos la librería [OpenNMT](https://opennmt.net/) para generar resumenes del tipo abstractivo.

Esta librería originalmente se especializó en modelos seq2seq para traducción. Sin embargo, también se puede ver el problema de summarization como una traducción a un texto más condensado, por lo que podemos usar los mismos modelos con este fin.

En este caso utilizaremos la versión para PyTorch de OpenNMT, `OpenNMT-py`. Esta nos probee de varias utilidades sobre PyTorch, que nos ayudaran en todo el proceso, partiendo por preprocesar los datos, para luego entrenar un modelo y finalmente evaluarlo y generar resumenes.

OpenNMT está pensado principalmente para ser ejecutado por medio de distintos scripts o comandos que procesan las distintas etapas. Los comandos son instalados junto con la librería y simplemente son una forma directa de acceder a los scripts. Sin emabargo, en el último tiempo han estado migrando esta forma de operar a una que sea más similar al uso de una librería, por esto, hay partes que su uso como tal todavía no parece natural.

Al ser pensada de esta forma, los archivos de configuración son bastante relevantes, para tener una referencia de cada parte del archivo de configuración, pueden ir a la [documentación de OpenNMT](https://opennmt.net/OpenNMT-py/examples/Summarization.html) para el script en particula que deseen usar.

A medida que vayamos avanzando en la actividad explicaremos los distintos componentes de esta librería que vayamos utilizando.


## Preámbulo

Primero debemos descargar distintas utilidades para realizar la actividad.

Debemos descargar el dataset hacia colab para luego decomprimirlo dentro del directorio `data`, que recién creamos. También importamos algunas de las librerías que ocuparemos durante esta actividad, como `rougue-score` para ayudarnos a calcular la métric ROUGE y `OpenNMT-py` nombrada anteriormente.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
! wget -q --show-progress https://raw.githubusercontent.com/Hernan4444/MyScripts/master/google_drive/google_api.py
! mkdir -p data/cnndm

google_api.py       100%[===================>]   5.55K  --.-KB/s    in 0s      


In [3]:
import os
from google_api import download_file_without_authenticate

# Descargamos nuestros checkpoints y archivos de configuración.
if not os.path.exists('lab4.zip'): 
    download_file_without_authenticate('18ecT1r9jFGyChATPwTdRgvj9dKctd3sS', 'lab4.zip') 

In [4]:
!unzip -nq lab4.zip -d . 
!mv lab4/activity\ 2/* .

In [5]:
# Descargamos los datos a utilizar.
!wget -nc https://s3.amazonaws.com/opennmt-models/Summary/cnndm.tar.gz -P data -q --show-progress
!tar -xf data/cnndm.tar.gz -C data/cnndm

cnndm.tar.gz        100%[===================>] 477.20M  14.1MB/s    in 36s     


In [6]:
# Instalamos los paquetes a utilizar.
!pip install rouge-score -q
!pip install git+https://github.com/OpenNMT/OpenNMT-py.git@905673188d78530246d40b77c9eac77bee909126 -q # Instalamos desde github pues queremos usar la versión en un commit específico.

     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 2.5MB 19.2MB/s 


## Creación del vocabulario

Antes de poder trabajar con datos de texto, debemos crear nuestro vocabulario y dejarlo fijo para esta aplicación, esto nos perimitirá transformar nuestras palabras (y tokens) a números. Con el siguiente comando la librería OpenNMT hará este trabajo por nosotros en base a la configuración que le demos en el archivo `cnndm.yaml`. Veamos su contenido:

En partícular son importantes los campos:
- `src_vocab`, `tgt_vocab`: Path a la ubicación donde se guardarán nuestros vobalarios creados.
- `share_vocab`: Si el vocabulario de src y tgt serán el mismo (notar que en traducción, por ejemplo, no se usa el mismo).
- `src_seq_length_trunc`, `tgt_seq_length_trunc`: Máximos largo de texto tanto para el input como el output.
- `data.cnndm.path_src`, `data.cnndm.path_tgt`: Corresponden al path de los datos de entrenamiento para el input y output respectivamente. `data.valid.<path>` contiene los datos de validación.

In [7]:
!cat configs/cnndm.yaml

save_data: data/cnndm
src_vocab: data/cnndm/run/cnndm.vocab.src
tgt_vocab: data/cnndm/run/cnndm.vocab.tgt
overwrite: False
src_seq_length_trunc: 400
tgt_seq_length_trunc: 100
share_vocab: True
data:
    cnndm:
        path_src: data/cnndm/train.txt.src
        path_tgt: data/cnndm/train.txt.tgt.tagged
    valid:
        path_src: data/cnndm/val.txt.src
        path_tgt: data/cnndm/val.txt.tgt.tagged


In [8]:
!onmt_build_vocab -config configs/cnndm.yaml -n_sample -1

Corpus cnndm's weight should be given. We default it to 1 for you.
[2021-01-08 12:47:09,542 INFO] Counter vocab from -1 samples.
[2021-01-08 12:47:09,543 INFO] n_sample=-1: Save full transformed corpus.
[2021-01-08 12:47:09,543 INFO] cnndm's transforms: TransformPipe()
[2021-01-08 12:47:09,543 INFO] Loading ParallelCorpus(data/cnndm/train.txt.src, data/cnndm/train.txt.tgt.tagged, align=None)...
[2021-01-08 12:47:09,662 WARNING] Empty line exists in cnndm#598.
[2021-01-08 12:47:09,665 WARNING] Empty line exists in cnndm#611.
[2021-01-08 12:47:09,677 WARNING] Empty line exists in cnndm#676.
[2021-01-08 12:47:09,680 WARNING] Empty line exists in cnndm#687.
[2021-01-08 12:47:09,739 WARNING] Empty line exists in cnndm#986.
[2021-01-08 12:47:09,802 WARNING] Empty line exists in cnndm#1276.
[2021-01-08 12:47:09,816 WARNING] Empty line exists in cnndm#1339.
[2021-01-08 12:47:09,864 WARNING] Empty line exists in cnndm#1561.
[2021-01-08 12:47:09,882 WARNING] Empty line exists in cnndm#1640.
[202

Finalmente, podemos observar el formato del vocabulario resultante y las entradas más comunes. Pueden observar que a la derecha del token en el vocabulario, aparece la frecuencia del término en el corpus (input y output) de dicho token.

In [9]:
!head data/cnndm/run/cnndm.vocab.src

.	12171859
the	11896137
,	9608868
to	5751070
a	5100465
and	4892178
of	4867755
in	4431027
's	2202677
was	2085975


## Entrenamiento

Al igual que en la parte 1 de este laboratorio, no entrenaremos un modelo sino que utilizaremos uno previamente entrenado. De todas formas en caso de que deseen entrenar su propi modelo pueden hacerlo tal y como se muestra a continuación.

También, como en la creación del vocabulario, entrenaremos el modelo utilizando un comando de OpenNMT, en este caso `onmt_train`. La configuración del entrenamiento se hace en base a archivos `.yaml` que se entrega al comando en la opción `-config`. 

En nuestro caso como tenemos tres modelos distintos existe un archivo por cada tipo de modelo:
1. Modelo base: `configs/config-cnndm-base.yaml`
2. Modelo que utiliza pointer networks: `configs/config-cnndm-pointer.yaml`
3. Modelo que utiliza pointer networks y coverage: `configs/config-cnndm-coverage.yaml`

A continuación pueden ver como ejemplo el archivo de configuración del modelo base:

In [10]:
!cat configs/config-cnndm-pointer.yaml

# maximum vocab size
src_vocab_size: 50000
tgt_vocab_size: 50000

# Corpus opts:
save_data: data/cnndm
src_vocab: data/cnndm/run/cnndm.vocab.src
tgt_vocab: data/cnndm/run/cnndm.vocab.src
overwrite: False
src_seq_length_trunc: 400
tgt_seq_length_trunc: 100
share_vocab: True
data:
    cnndm:
        path_src: data/cnndm/train.txt.src
        path_tgt: data/cnndm/train.txt.tgt.tagged
    valid:
        path_src: data/cnndm/val.txt.src
        path_tgt: data/cnndm/val.txt.tgt.tagged

save_model: checkpoints/base/checkpoint
global_attention: mlp
word_vec_size: 128
rnn_size: 512
layers: 2
encoder_type: brnn
train_steps: 200000
max_grad_norm: 2
dropout: 0
batch_size: 16
valid_batch_size: 16
optim: adagrad
learning_rate: 0.15
adagrad_accumulator_init: 0.1
copy_attn: true
reuse_copy_attn: true
coverage_attn: false
lambda_coverage: 0.0
copy_loss_by_seqlength: true
bridge: true
seed: 777
world_size: 1
gpu_ranks: [0]

El archivo de configuración sigue los mismo principios que el anterior y las llaves que se repiten tienen el mismo significado. Para el entrnamiento destacamos:

- `src_vocab_size`: El tamaño del vocabulario a utilizar.
- `save_model`: Lugar donde se guardaran los checkpoints durante el entrenamiento.
- `copy_attn`: `True` para utilizar pointer networks, `False` en caso contrario.
- `coverage_attn`: `True` para utilizar el módulo de coverages, `False` en caso contrario.
- `lambda_coverage`: En caso de utilizar coverage, este parámetro define cuanto se ponderará en la pérdida final.

En caso de querer entrenar un modelo propio, solo deben descomentar las siguientes celdas de esta sección. Si desean cambiar el tipo de modelo utilizado, distinto del base, deben setear el path al archivo de configuración correcto en la variable `CONFIG` para poder realizar el entrenamiento.

In [11]:
# %%shell
# CONFIG=configs/config-cnndm-base.yaml
# onmt_train -config $CONFIG
# ls checkpoints/base/

**Importante:** En caso de querer usar el modelo entrenado para los siguientes pasos de la actividad, debes setear correctamente tanto el archivo de configuración usado como el checkpoint deseado cuando sea necesario a continuación.

El archivo de checkpoint debe ser el mismo que el seteado en la variable de entorno `$CONFIG`. 

Y el checkpoint estará en la carpeta `checkpoints/base/`, ahí el valor que aparece en el nombre corresponde a la iteración en que fue creado, se recomienda usar el último.

## Creación de Resumenes
En esta sección veremos como funcionan los tres modelos pre-entrenados de forma cualitativa y cuantitativamente.

Dado que queremos tener más flexibilidad en este proceso, utilizaremos OpenNMT como librería, importando y utilizando sus distintos módulos.

### Preámbulo

Primero debemos importar los paquetes y módulos a utilizar. También definiremos funciones auxiliares que nos ayudaran durante la ejecución de esta parte del laboratorio.

In [12]:
from collections import defaultdict, Counter
import random

import torch
from torch import nn
from tqdm import tqdm_notebook as tqdm
from rouge_score.scoring import BootstrapAggregator

import onmt
from onmt.inputters.inputter import _load_vocab, _build_fields_vocab, get_fields
from onmt.model_builder import build_model
from onmt.models.model_saver import load_checkpoint
from onmt.translate import GNMTGlobalScorer, Translator, TranslationBuilder
from onmt.opts import dynamic_prepare_opts, translate_opts
from onmt.utils.parse import ArgumentParser

In [13]:
# Obtención del checkpoint en base a su path
def get_checkpoint_data(checkpoint_path):
    checkpoint = load_checkpoint(checkpoint_path)
    fields = checkpoint['vocab']
    opts = checkpoint['opt']
    return checkpoint, fields, opts

# Actualización de la configuración por defecto en base a las del checkpoint
def update_opts(opts, config_path):
    parser = ArgumentParser(description='translate.py')
    dynamic_prepare_opts(parser, build_vocab_only=False)
    translate_opts(parser)

    base_args = ['-config', config_path,
                 '-out_file', './out.txt'
                 '-n_sample', '-1',
                 '-gpu', '0',
                 '-batch_size', '20',
                 '-beam_size', '10',
                 '-model', 'checkpoints/ptr_checkpoint.pt',
                 '-src', 'data/cnndm/test.txt.src',
                 '-output', 'testout/cnndm.out',
                 '-min_length', '35',
                 '-verbose', 
                 '-stepwise_penalty', 
                 '-coverage_penalty', 'summary',
                 '-beta', '5',
                 '-length_penalty', 'wu',
                 '-alpha', '0.9',
                 '-verbose', 
                 '-block_ngram_repeat', '3',
                 '-ignore_when_blocking', '"."', '"</t>" "<t>"']

    opts, unknown = parser.parse_known_args(base_args, namespace=opts)

    model_opt = ArgumentParser.ckpt_model_opts(opts)
    ArgumentParser.update_model_opts(model_opt)
    ArgumentParser.validate_model_opts(model_opt)

    opts.out_file = './out.txt'
    return opts, model_opt

Además, crearemos varias funciones que nos ayudarán el entrenaminto de modelos reiteradas veces en este práctico. Les recomendamos que traten de entender cada una de estas funciones pues podrán reforzar conocimientos que ya han aprendido en el diplomado hasta ahora, pero para efectos de esta actividad no es necesario hacerlo.

In [14]:
import re
import textwrap

from rouge_score import rouge_scorer

# Utilidades

def normalize_txt(txt):
    norm_str = txt
    norm_str = norm_str.replace('</t> <t>', '\n')
    norm_str = re.sub('</?t>', '', norm_str)
    norm_str = re.sub(' +', ' ', norm_str)
    norm_str = re.sub('\n ', '\n', norm_str)
    norm_str = norm_str.strip()
    return norm_str

def translation2texts(translation):
    src = normalize_txt(' '.join(translation.src_raw))
    tgt = normalize_txt(' '.join(translation.gold_sent))
    preds = [normalize_txt(' '.join(sents)) for sents in translation.pred_sents]
    return src, tgt, preds


# Funciones auxiliares para mostrar los resultados

def bold(text):
  return '\033[1m' + text + '\033[0m'

def display_translation(translation):
    src, tgt, preds = translation2texts(translation)

    print(bold('Source Text:'))
    print(textwrap.fill(src, width=70), end='\n\n')

    print(bold('Gold Standard Summarization:'))
    print(textwrap.fill(tgt, width=70), end='\n\n')

    print(bold('Predicted Summarization:'))
    for sent in preds:
        print(textwrap.fill(sent, width=70), end='\n\n')

# Función auxiliares para calcular el ROUGE en base a una `Translation`
def translation_score(translation):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeLsum'], use_stemmer=False)
    src, tgt, preds = translation2texts(translation)
    scores = scorer.score(tgt, preds[0])
    return scores

Las siguientes dos funciones son interesantes pues se utilizan para crear el `Dataset` (`build_dataset`) y el `Translator` (`build_translator`).

El `Dataset` al igual que los que ya han visto al utilizar PyTorch, nos ayudará a acceder los datos y dejarlos en el formato correcto para el modelo.

Por otro lado, el `Translator` nos proveerá funcionalidades necesarias para poder obtener "traducciones" (`Translations`), en nuestro caso los resumenes, a partir de un batch de datos. En particular nos facilitará mucho el trabajo de hacer el decoding de lo predicho por el modelo y formar así el texto resultante. Nos ofrece dos funciones para este propósito, `Translator.translate` y `Translator.translate_batch` que reciben un `src` que será traducido, o resumido en nuestro caso. 

El resultado de estos métodos lo podemos pasar por un `TranslationBuilder` el cual transformará de indices de tokens a texto y nos retornará una `Translation`. Esta última es una estructura de datos que agrupa información relevante a una traducción.
Algunos atributos importantes son:
- `src`: Input en forma de ids de tokens.
- `src_raw`: Input en forma de texto plano.
- `pred_sents`: Output en forma de ids de tokens.
- `pred_scores`: Output en forma de texto plano.
- `attns`: Attenciones del modelo
- `gold_sent`: El output de referencia del resumen (*ground truth*).




Notar que el `Translator` no corresponde a un modelo de PyTorch. Como no usaremos esta clase para entrenar no es necesario que lo sea, así que simplemente utiliza un modelo que le debemos entregar con este proposito. Esto es bueno pues de desacopla del tipo de modelo utilizado sin afectar su forma de funcionar.

In [15]:
def build_dataset(src_path, tgt_path):
    src_data = {"reader": onmt.inputters.str2reader["text"].from_opt(opts), "data": src_path}
    tgt_data = {"reader": onmt.inputters.str2reader["text"].from_opt(opts), "data": tgt_path}
    _readers, _data = onmt.inputters.Dataset.config(
        [('src', src_data), ('tgt', tgt_data)])

    dataset = onmt.inputters.Dataset(
        fields, readers=_readers, data=_data,
        sort_key=onmt.inputters.str2sortkey["text"])

    return dataset

def build_translator(opts, model, dataset, fields, tgt_path):
    scorer = onmt.translate.GNMTGlobalScorer.from_opt(opts)

    translator = Translator.from_opt(
        model,
        fields,
        opts,
        model_opt,
        global_scorer=scorer,
        out_file=opts.out_file,
        report_align=False,
        report_score=True
    )

    builder = onmt.translate.TranslationBuilder(
        dataset, fields, opts.n_best, opts.replace_unk, tgt_path,
        opts.phrase_table
    )

    return translator, scorer, builder

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

### Modelo Base

Habiendo explorados los datos, prosigamos a explorar un modelo base. Este modelo nos servirá para poder tener un punto de comparación, y así tener una mejor idea de cuanto ayuda usar las técnicas que vimos en clases.

Este modelo consiste en una arquitectura seq2seq con atención basado en LSTMs bidireccionales, igual al modelo base que vimos en clase.

#### Configuración de la ejecución

Para poder configurar correctamente nuestra ejecución utlizaremos los argumentos previamente definidos, para que estos estén ad-hoc a como está dispuesto nuestro entorno de colab. También consideraremos los argumentos que se usaron en el entrenamiento para mantenerlos igual en la creación del modelo.

Estos últimos vienen en el checkpoint que tenemos del entrenamiento, así que lo leemos y actualizamos nuestros argumentos.

In [ ]:
checkpoint_path = 'checkpoints/base_checkpoint.pt'
config_path = 'configs/config-cnndm-base.yaml'

checkpoint, fields, opts = get_checkpoint_data(checkpoint_path)
opts, model_opt = update_opts(opts, config_path)

Debemos crear nuestro `Dataset` y también una especie de `DataLoader`, que en este caso es un `torchtext.data.Iterator` y nos ayudará a recorrer nuestros datos.

In [ ]:
src_val = "data/cnndm/val.txt.src"
tgt_val = "data/cnndm/val.txt.tgt.tagged"
src_test = "data/cnndm/test.txt.src"
tgt_test = "data/cnndm/test.txt.tgt.tagged"

src = src_test
tgt = tgt_test

dataset = build_dataset(src, tgt)
data_iter = onmt.inputters.OrderedIterator(
                dataset=dataset,
                device=device,
                batch_size=opts.batch_size,
                train=False,
                sort=False,
                sort_within_batch=True,
                shuffle=False,
            )

Debemos crear nuestro modelo en base la configuración que tenemos para nuestro modelo base.

In [ ]:
model = build_model(model_opt, opts, fields, checkpoint)
model.to(device)

translator, scorer, builder = build_translator(opts, model, dataset, fields, tgt)

#### Evaluación cualitativa

Veamos algunos de los ejemplos abstractivos que produce nuestro modelo. Podemos aprovechar funciones que nos da `OpenNMT` para realizar esta inspección.

In [ ]:
for batch in data_iter:
    trans_batch = translator.translate_batch(
        batch=batch, src_vocabs=dataset.src_vocabs,
        attn_debug=False)
    translations = builder.from_batch(trans_batch)
    for trans in translations:
        print(trans.log(0))
    break


SENT 0: ['marseille', ',', 'france', '-lrb-', 'cnn', '-rrb-', 'the', 'french', 'prosecutor', 'leading', 'an', 'investigation', 'into', 'the', 'crash', 'of', 'germanwings', 'flight', '9525', 'insisted', 'wednesday', 'that', 'he', 'was', 'not', 'aware', 'of', 'any', 'video', 'footage', 'from', 'on', 'board', 'the', 'plane', '.', 'marseille', 'prosecutor', 'brice', 'robin', 'told', 'cnn', 'that', '``', 'so', 'far', 'no', 'videos', 'were', 'used', 'in', 'the', 'crash', 'investigation', '.', "''", 'he', 'added', ',', '``', 'a', 'person', 'who', 'has', 'such', 'a', 'video', 'needs', 'to', 'immediately', 'give', 'it', 'to', 'the', 'investigators', '.', "''", 'robin', "'s", 'comments', 'follow', 'claims', 'by', 'two', 'magazines', ',', 'german', 'daily', 'bild', 'and', 'french', 'paris', 'match', ',', 'of', 'a', 'cell', 'phone', 'video', 'showing', 'the', 'harrowing', 'final', 'seconds', 'from', 'on', 'board', 'germanwings', 'flight', '9525', 'as', 'it', 'crashed', 'into', 'the', 'french', 'a

Sin embargo estas no son muy legibles como pueden ver previamente, así que utilizaremos también una función propia en donde seleccionaremos al azar uno de estos ejemplos y la miraremos en más detalle.

In [ ]:
random_translation = random.choice(translations)
display_translation(random_translation)

Source Text:
-lrb- cnn -rrb- never mind cats having nine lives . a stray pooch in
washington state has used up at least three of her own after being hit
by a car , apparently whacked on the head with a hammer in a misguided
mercy killing and then buried in a field -- only to survive . that 's
according to washington state university , where the dog -- a friendly
white-and-black bully breed mix now named theia -- has been receiving
care at the veterinary teaching hospital . four days after her
apparent death , the dog managed to stagger to a nearby farm , dirt-
covered and emaciated , where she was found by a worker who took her
to a vet for help . she was taken in by moses lake , washington ,
resident sara mellado . `` considering everything that she 's been
through , she 's incredibly gentle and loving , '' mellado said ,
according to wsu news . `` she 's a true miracle dog and she deserves
a good life . '' theia is only one year old but the dog 's brush with
death did not leave her u

#### Evaluación cuantitativa

Ya habiendo visto ejemplos de las predicciones que genera nuestro modelo, podemos calcular también una métrica de rendimiento basada en ROUGE.

Primero calculemos cuanto es el ROUGE para el ejemplo particular visto previamente.

In [ ]:
src, tgt, preds = translation2texts(random_translation)
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeLsum'], use_stemmer=False)
scores = scorer.score(tgt, preds[0])
for score_type in scores:
    score = scores[score_type].fmeasure
    print(f'{score_type:10.10s}: {score:.4f}')

rouge1    : 0.4324
rouge2    : 0.1389
rougeLsum : 0.4324


Finalmente calculemos la métrica ROUGE para el set de test. Calcularla toma un buen rato (una hora aproximadamente) por lo que solo utilizaremos el 10% del set de test y así tener una mejor idea de como estea funcionando nuestro generador de resumenes.

In [ ]:
aggregator = BootstrapAggregator()

total = len(data_iter)

# calcularemos el ROUGE sobre el 10% del set de test solamente, cambiar a 1 para el set completo
test_ratio = 0.1
stop = int(test_ratio * total)

for i, batch in tqdm(enumerate(data_iter), total=min(stop, total)):
    if i > stop:
        break
    trans_batch = translator.translate_batch(
        batch=batch, src_vocabs=dataset.src_vocabs,
        attn_debug=False)
    translations = builder.from_batch(trans_batch)

    for translation in translations:
        score = translation_score(translation)
        aggregator.add_scores(score)

result = aggregator.aggregate()

for score_type in result:
    score = result[score_type].mid.fmeasure
    print(f'{score_type:10.10s}: {score:.4f}')

rouge1    : 0.2631
rouge2    : 0.0846
rougeLsum : 0.2379


### Modelo Pointer

In [ ]:
checkpoint_path = 'checkpoints/ptr_checkpoint.pt'
config_path = 'configs/config-cnndm-pointer.yaml'

checkpoint, fields, opts = get_checkpoint_data(checkpoint_path)
opts, model_opt = update_opts(opts, config_path)

In [ ]:
src_val = "data/cnndm/val.txt.src"
tgt_val = "data/cnndm/val.txt.tgt.tagged"
src_test = "data/cnndm/test.txt.src"
tgt_test = "data/cnndm/test.txt.tgt.tagged"

src = src_test
tgt = tgt_test

dataset = build_dataset(src, tgt)
data_iter = onmt.inputters.OrderedIterator(
                dataset=dataset,
                device=device,
                batch_size=opts.batch_size,
                train=False,
                sort=False,
                sort_within_batch=True,
                shuffle=False,
            )

Debemos crear nuestro modelo en base la configuración que tenemos para nuestro modelo base.

In [ ]:
model = build_model(model_opt, opts, fields, checkpoint)
model.to(device)

translator, scorer, builder =build_translator(opts, model, dataset, fields, tgt)

#### Evaluación cualitativa

Veamos algunos de los ejemplos abstractivos que produce nuestro modelo. Podemos aprovechar funciones que nos da `OpenNMT` para realizar esta inspección.

In [ ]:
for batch in data_iter:
    trans_batch = translator.translate_batch(
        batch=batch, src_vocabs=dataset.src_vocabs,
        attn_debug=False)
    translations = builder.from_batch(trans_batch)
    for trans in translations:
        print(trans.log(0))
    break


SENT 0: ['marseille', ',', 'france', '-lrb-', 'cnn', '-rrb-', 'the', 'french', 'prosecutor', 'leading', 'an', 'investigation', 'into', 'the', 'crash', 'of', 'germanwings', 'flight', '9525', 'insisted', 'wednesday', 'that', 'he', 'was', 'not', 'aware', 'of', 'any', 'video', 'footage', 'from', 'on', 'board', 'the', 'plane', '.', 'marseille', 'prosecutor', 'brice', 'robin', 'told', 'cnn', 'that', '``', 'so', 'far', 'no', 'videos', 'were', 'used', 'in', 'the', 'crash', 'investigation', '.', "''", 'he', 'added', ',', '``', 'a', 'person', 'who', 'has', 'such', 'a', 'video', 'needs', 'to', 'immediately', 'give', 'it', 'to', 'the', 'investigators', '.', "''", 'robin', "'s", 'comments', 'follow', 'claims', 'by', 'two', 'magazines', ',', 'german', 'daily', 'bild', 'and', 'french', 'paris', 'match', ',', 'of', 'a', 'cell', 'phone', 'video', 'showing', 'the', 'harrowing', 'final', 'seconds', 'from', 'on', 'board', 'germanwings', 'flight', '9525', 'as', 'it', 'crashed', 'into', 'the', 'french', 'a

Sin embargo estas no son muy legibles como pueden ver previamente, así que utilizaremos también una función propia en donde seleccionaremos al azar uno de estos ejemplos y lo miraremos más en detalle.

In [ ]:
random_translation = random.choice(translations)
display_translation(random_translation)

Source Text:
-lrb- cnn -rrb- never mind cats having nine lives . a stray pooch in
washington state has used up at least three of her own after being hit
by a car , apparently whacked on the head with a hammer in a misguided
mercy killing and then buried in a field -- only to survive . that 's
according to washington state university , where the dog -- a friendly
white-and-black bully breed mix now named theia -- has been receiving
care at the veterinary teaching hospital . four days after her
apparent death , the dog managed to stagger to a nearby farm , dirt-
covered and emaciated , where she was found by a worker who took her
to a vet for help . she was taken in by moses lake , washington ,
resident sara mellado . `` considering everything that she 's been
through , she 's incredibly gentle and loving , '' mellado said ,
according to wsu news . `` she 's a true miracle dog and she deserves
a good life . '' theia is only one year old but the dog 's brush with
death did not leave her u

#### Evaluación cuantitativa

Ya habiendo visto ejemplos de las predicciones que genera nuestro modelo, podemos calcular también una métrica de rendimineto basada en ROUGE.

Primero calculemos cuanto es el ROUGE para el ejemplo particular visto previamente.

In [ ]:
src, tgt, preds = translation2texts(random_translation)
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeLsum'], use_stemmer=False)
scores = scorer.score(tgt, preds[0])
for score_type in scores:
    score = scores[score_type].fmeasure
    print(f'{score_type:10.10s}: {score:.4f}')

rouge1    : 0.2588
rouge2    : 0.0723
rougeLsum : 0.2353


Finalmente calculemos la métrica ROUGE para el set de test. Calcularla toma un buen rato (una hora aproximadamente) por lo que solo utilizaremos el 10% del set de test y así tener una mejor idea de como estea funcionando nuestro generador de resumenes.

In [ ]:
aggregator = BootstrapAggregator()

total = len(data_iter)

# calcularemos el ROUGE sobre el 10% del set de test solamente, cambiar a 1 para el set completo
test_ratio = 0.1
stop = int(test_ratio * total)

for i, batch in tqdm(enumerate(data_iter), total=min(stop, total)):
    if i > stop:
        break
    trans_batch = translator.translate_batch(
        batch=batch, src_vocabs=dataset.src_vocabs,
        attn_debug=False)
    translations = builder.from_batch(trans_batch)

    for translation in translations:
        score = translation_score(translation)
        aggregator.add_scores(score)

result = aggregator.aggregate()

for score_type in result:
    score = result[score_type].mid.fmeasure
    print(f'{score_type:10.10s}: {score:.4f}')

rouge1    : 0.2724
rouge2    : 0.0957
rougeLsum : 0.2439


#### Actividad 2

A continuación responda las siguientes preguntas.
1. ¿Qué diferencia (una) cualitativa nota entre las predicciones del model base con el que usa pointer networks?
2. ¿Qué modelo alcanzó mejor rendimiento en ROUGE? Responda para las tres métrias de ROUGE distintas.

**Respuestas:**

1.

In [ ]:
Diferencias = "Se observa que el modelo con pointer tiene menos vectores unknown que el modelo base" #@param {type:"string"}

2.

En los tres tipos de rouge el modelo pointer tiene mayores valores que el modelo base

In [ ]:
rouge_1 = "modelo base= 0.2631; modelo pointer = 0.2724" #@param {type:"string"}
rouge_2 = "modelo base= 0.0846; modelo pointer = 0.0957" #@param {type:"string"}
rouge_L = "modelo base= 0.2379; modelo pointer = 0.2439" #@param {type:"string"}

### Actividad 3: Modelo Coverage

Descomente y ejecute la evaluación cualitativa y cuantitativa del modelo de coverage. Reporte al final los resultados obtenidos en la evaluación cuantitativa.

**Ayuda:** No debe modificar el código, pero si fijarse de setear los archivos de configuración correctos correspondientes.

In [17]:
checkpoint_path = 'checkpoints/coverage_checkpoint.pt'
config_path = 'configs/config-cnndm-coverage.yaml'

checkpoint, fields, opts = get_checkpoint_data(checkpoint_path)
opts, model_opt = update_opts(opts, config_path)

In [18]:
src_val = "data/cnndm/val.txt.src"
tgt_val = "data/cnndm/val.txt.tgt.tagged"
src_test = "data/cnndm/test.txt.src"
tgt_test = "data/cnndm/test.txt.tgt.tagged"

src = src_test
tgt = tgt_test

dataset = build_dataset(src, tgt)
data_iter = onmt.inputters.OrderedIterator(
                dataset=dataset,
                device=device,
                batch_size=opts.batch_size,
                train=False,
                sort=False,
                sort_within_batch=True,
                shuffle=False,
            )

Debemos crear nuestro modelo en base la configuración que tenemos para nuestro modelo base.

In [19]:
model = build_model(model_opt, opts, fields, checkpoint)
model.to(device)

translator, scorer, builder =build_translator(opts, model, dataset, fields, tgt)

#### Evaluación cualitativa

Veamos algunos de los ejemplos abstractivos que produce nuestro modelo. Podemos aprovechar funciones que nos da `OpenNMT` para realizar esta inspección.

In [20]:
for batch in data_iter:
    trans_batch = translator.translate_batch(
        batch=batch, src_vocabs=dataset.src_vocabs,
        attn_debug=False)
    translations = builder.from_batch(trans_batch)
    for trans in translations:
        print(trans.log(0))
    break


SENT 0: ['marseille', ',', 'france', '-lrb-', 'cnn', '-rrb-', 'the', 'french', 'prosecutor', 'leading', 'an', 'investigation', 'into', 'the', 'crash', 'of', 'germanwings', 'flight', '9525', 'insisted', 'wednesday', 'that', 'he', 'was', 'not', 'aware', 'of', 'any', 'video', 'footage', 'from', 'on', 'board', 'the', 'plane', '.', 'marseille', 'prosecutor', 'brice', 'robin', 'told', 'cnn', 'that', '``', 'so', 'far', 'no', 'videos', 'were', 'used', 'in', 'the', 'crash', 'investigation', '.', "''", 'he', 'added', ',', '``', 'a', 'person', 'who', 'has', 'such', 'a', 'video', 'needs', 'to', 'immediately', 'give', 'it', 'to', 'the', 'investigators', '.', "''", 'robin', "'s", 'comments', 'follow', 'claims', 'by', 'two', 'magazines', ',', 'german', 'daily', 'bild', 'and', 'french', 'paris', 'match', ',', 'of', 'a', 'cell', 'phone', 'video', 'showing', 'the', 'harrowing', 'final', 'seconds', 'from', 'on', 'board', 'germanwings', 'flight', '9525', 'as', 'it', 'crashed', 'into', 'the', 'french', 'a

Sin embargo estas no son muy legibles como pueden ver previamente, así que utilizaremos también una función propia en donde seleccionaremos al azar uno de estos ejemplos y lo miraremos más en detalle.

In [21]:
random_translation = random.choice(translations)
display_translation(random_translation)

Source Text:
-lrb- cnn -rrb- never mind cats having nine lives . a stray pooch in
washington state has used up at least three of her own after being hit
by a car , apparently whacked on the head with a hammer in a misguided
mercy killing and then buried in a field -- only to survive . that 's
according to washington state university , where the dog -- a friendly
white-and-black bully breed mix now named theia -- has been receiving
care at the veterinary teaching hospital . four days after her
apparent death , the dog managed to stagger to a nearby farm , dirt-
covered and emaciated , where she was found by a worker who took her
to a vet for help . she was taken in by moses lake , washington ,
resident sara mellado . `` considering everything that she 's been
through , she 's incredibly gentle and loving , '' mellado said ,
according to wsu news . `` she 's a true miracle dog and she deserves
a good life . '' theia is only one year old but the dog 's brush with
death did not leave her u

#### Evaluación cuantitativa

Ya habiendo visto ejemplos de las predicciones que genera nuestro modelo, podemos calcular también una métrica de rendimiento basada en ROUGE.

Primero calculemos cuanto es el ROUGE para el ejemplo particular visto previamente.

In [22]:
src, tgt, preds = translation2texts(random_translation)
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeLsum'], use_stemmer=False)
scores = scorer.score(tgt, preds[0])
for score_type in scores:
    score = scores[score_type].fmeasure
    print(f'{score_type:10.10s}: {score:.4f}')

rouge1    : 0.2588
rouge2    : 0.0723
rougeLsum : 0.2353


Finalmente calculemos la métrica ROUGE para el set de test. Calcularla toma un buen rato (una hora aproximadamente) por lo que solo utilizaremos el 10% del set de test y así tener una mejor idea de como está funcionando nuestro generador de resumenes.

In [23]:
aggregator = BootstrapAggregator()

total = len(data_iter)

# calcularemos el ROUGE sobre el 10% del set de test solamente, cambiar a 1 para el set completo
test_ratio = 0.1
stop = int(test_ratio * total)

for i, batch in tqdm(enumerate(data_iter), total=min(stop, total)):
    if i > stop:
        break
    trans_batch = translator.translate_batch(
        batch=batch, src_vocabs=dataset.src_vocabs,
        attn_debug=False)
    translations = builder.from_batch(trans_batch)

    for translation in translations:
        score = translation_score(translation)
        aggregator.add_scores(score)

result = aggregator.aggregate()

for score_type in result:
    score = result[score_type].mid.fmeasure
    print(f'{score_type:10.10s}: {score:.4f}')

rouge1    : 0.2705
rouge2    : 0.0943
rougeLsum : 0.2433


#### Respuesta Actividad 3

In [ ]:
rouge_1 = 0.2705 #@param {type:"number"}
rouge_2 = 0.0943 #@param {type:"number"}
rouge_L = 0.2433 #@param {type:"number"}

Se observa que estos valores de rouge son levemente menores al del modelo de pointer, pero mayores que el modelo base